In [1]:
import os
import json
import pandas as pd
from keplergl import KeplerGl

INPUT_FILE = "Location History.json"
OUTPUT_FILE = "google_loc_data.parquet"
OUTPUT_HTML_FILE = "kepler_gl_map.html"

In [2]:
with open(INPUT_FILE, "r") as file:
    input_json = json.load(file)

In [3]:
print("No:of points - " + str(len(input_json['locations'])))

No:of points - 561400


In [4]:
df = pd.DataFrame(input_json['locations'])

In [5]:
df["datetime"] = pd.to_datetime(df["timestampMs"], unit="ms")
df.drop(columns=['accuracy', 'velocity', 'heading', 'verticalAccuracy', 'activity', 'timestampMs'], inplace=True)
df.rename(columns={'latitudeE7': 'lat'}, inplace=True)
df.rename(columns={'longitudeE7': 'lon'}, inplace=True)
df['lat'] = df['lat'].div(10000000)
df['lon'] = df['lon'].div(10000000)

In [6]:
df.to_parquet(OUTPUT_FILE, compression='brotli')


# Reformat datetime to string for JSON serialization
df['datetime'] = df['datetime'].astype(str)

In [7]:
config = {
    'version': 'v1',
    'config': {
        'mapState': {
            'latitude': df.lat.median(),
            'longitude': df.lon.median(),
            'zoom': 5
        }
    }
}

keplermap = KeplerGl(height=900, config=config)

# Add the data
keplermap.add_data(data=df, name='data1')

keplermap

User Guide: https://github.com/keplergl/kepler.gl/blob/master/docs/keplergl-jupyter/user-guide.md


KeplerGl(config={'version': 'v1', 'config': {'mapState': {'latitude': 11.1026466, 'longitude': 78.6762159, 'zo…

In [8]:
# save current map
keplermap.save_to_html(file_name=OUTPUT_HTML_FILE)

Map saved to kepler_gl_map.html!
